In [115]:
!pip install folium
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import json
import folium # map rendering library
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np

pd.set_option('display.max.columns',None)
pd.set_option('display.max.rows',None)

In [116]:
#retrieve URL and find tables to extract data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page,'lxml')
all_tables = soup.find_all('table')
right_tables = soup.find('table',class_='wikitable sortable')

In [117]:
#populate dataframe
A = [] #column 1 - postal code
B = [] #column 2 - borough
C = [] #column 3 - neighborhood

for row in right_tables.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df = pd.DataFrame(A, columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

In [118]:
#strip data of extra spaces and \n
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [119]:
#exclude rows that have "Not assigned" in the column "Borough"
df_clean = df[df['Borough']!='Not assigned']

In [120]:
df_clean.shape

(103, 3)

In [121]:
df_clean.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [122]:
# The code was removed by Watson Studio for sharing.

In [123]:
df_clean2 = pd.merge(df,geo)

In [124]:
df_clean2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [125]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [126]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_clean2['Latitude'], df_clean2['Longitude'], df_clean2['Borough'], df_clean2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

I chose the borough North York because it has the highest number of neighborhoods

In [127]:
northyork_data = df_clean2[df_clean2['Borough']=='North York'].reset_index(drop=True)
northyork_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [128]:
address = 'North York, Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [129]:
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)
    
map_northyork

In [130]:
CLIENT_ID = 'A25BUAI2EZ1UTNKDQHHGSGOSWGAVIW52DKFEFTS514K1WNPL' # your Foursquare ID
CLIENT_SECRET = 'BUZHUIRQP33IYXNVWWN33SGCAQMZMA2CJ0CVLJQHFLMAOHAW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials:
CLIENT_ID: A25BUAI2EZ1UTNKDQHHGSGOSWGAVIW52DKFEFTS514K1WNPL
CLIENT_SECRET:BUZHUIRQP33IYXNVWWN33SGCAQMZMA2CJ0CVLJQHFLMAOHAW


In [131]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [132]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=A25BUAI2EZ1UTNKDQHHGSGOSWGAVIW52DKFEFTS514K1WNPL&client_secret=BUZHUIRQP33IYXNVWWN33SGCAQMZMA2CJ0CVLJQHFLMAOHAW&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [133]:
results = requests.get(url).json()

In [134]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [136]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [137]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [138]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [139]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Don Mills,25,25,25,25,25,25
Downsview,15,15,15,15,15,15
"Fairview, Henry Farm, Oriole",69,69,69,69,69,69
Glencairn,6,6,6,6,6,6
Hillcrest Village,5,5,5,5,5,5
Humber Summit,3,3,3,3,3,3


In [140]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [141]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()

In [142]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.10
1     Coffee Shop  0.10
2     Gas Station  0.05
3  Sandwich Place  0.05
4       Gift Shop  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4       Lingerie Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3         Pizza Place  0.05
4      Breakfast Spot  0.05


----Don Mills----
                 venue  freq
0                  Gym  0.12
1           Beer Store  0.08
2  Japanese Restaurant  0.08
3                 Café  0.08
4           Restaurant  0.08


----Downsview----
            venue  freq
0   Grocery Store  0.20
1            Park  0.13
2            Bank  0.07
3        Bus Stop  0.07
4  Discount Store  0.07


----Fairview, Henry Farm, Or

In [154]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Park,Pharmacy,Pizza Place,Bridal Shop,Deli / Bodega,Ice Cream Shop,Restaurant,Gift Shop
1,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Women's Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Pub,Restaurant,Juice Bar,Café,Butcher,Indian Restaurant,Comfort Food Restaurant
3,Don Mills,Gym,Beer Store,Café,Japanese Restaurant,Restaurant,Coffee Shop,Art Gallery,Sandwich Place,Athletics & Sports,Dim Sum Restaurant
4,Downsview,Grocery Store,Park,Food Truck,Bank,Liquor Store,Bus Stop,Hotel,Baseball Field,Shopping Mall,Discount Store


In [156]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 4, 1, 2, 0, 0, 2],
      dtype=int32)

In [186]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge northyork_grouped with northyork_data to add latitude/longitude for each neighborhood
northyork_merged.reset_index()
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Fast Food Restaurant,Department Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Women's Store,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Sporting Goods Shop,Boutique,Carpet Store,Coffee Shop,Event Space,Vietnamese Restaurant,Accessories Store,Steakhouse
3,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Gym,Beer Store,Café,Japanese Restaurant,Restaurant,Coffee Shop,Art Gallery,Sandwich Place,Athletics & Sports,Dim Sum Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Pizza Place,Sushi Restaurant,Bakery,Japanese Restaurant,Pub,Women's Store,Department Store,Chocolate Shop,Clothing Store,Coffee Shop
5,M3C,North York,Don Mills,43.725900,-79.340923,0.0,Gym,Beer Store,Café,Japanese Restaurant,Restaurant,Coffee Shop,Art Gallery,Sandwich Place,Athletics & Sports,Dim Sum Restaurant
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Fast Food Restaurant,Golf Course,Mediterranean Restaurant,Dog Run,Pool,Women's Store,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0.0,Bank,Coffee Shop,Park,Pharmacy,Pizza Place,Bridal Shop,Deli / Bodega,Ice Cream Shop,Restaurant,Gift Shop
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Shoe Store,Restaurant,Juice Bar,Bank,Bakery,Women's Store
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0.0,Furniture / Home Store,Caribbean Restaurant,Massage Studio,Falafel Restaurant,Coffee Shop,Bar,Miscellaneous Shop,Department Store,Clothing Store,Comfort Food Restaurant


In [194]:
northyork_merged.dropna(inplace=True)
northyork_merged.Cluster_Labels = northyork_merged.Cluster_Labels.astype(int)

In [188]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [197]:
northyork_merged.loc[northyork_merged['Cluster_Labels'] == 0, northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Lawrence Manor, Lawrence Heights",0,Clothing Store,Furniture / Home Store,Sporting Goods Shop,Boutique,Carpet Store,Coffee Shop,Event Space,Vietnamese Restaurant,Accessories Store,Steakhouse
3,Don Mills,0,Gym,Beer Store,Café,Japanese Restaurant,Restaurant,Coffee Shop,Art Gallery,Sandwich Place,Athletics & Sports,Dim Sum Restaurant
5,Don Mills,0,Gym,Beer Store,Café,Japanese Restaurant,Restaurant,Coffee Shop,Art Gallery,Sandwich Place,Athletics & Sports,Dim Sum Restaurant
6,Hillcrest Village,0,Fast Food Restaurant,Golf Course,Mediterranean Restaurant,Dog Run,Pool,Women's Store,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop
7,"Bathurst Manor, Wilson Heights, Downsview North",0,Bank,Coffee Shop,Park,Pharmacy,Pizza Place,Bridal Shop,Deli / Bodega,Ice Cream Shop,Restaurant,Gift Shop
8,"Fairview, Henry Farm, Oriole",0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Shoe Store,Restaurant,Juice Bar,Bank,Bakery,Women's Store
9,"Northwood Park, York University",0,Furniture / Home Store,Caribbean Restaurant,Massage Studio,Falafel Restaurant,Coffee Shop,Bar,Miscellaneous Shop,Department Store,Clothing Store,Comfort Food Restaurant
10,Bayview Village,0,Japanese Restaurant,Café,Bank,Chinese Restaurant,Women's Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
11,Downsview,0,Grocery Store,Park,Food Truck,Bank,Liquor Store,Bus Stop,Hotel,Baseball Field,Shopping Mall,Discount Store
13,Downsview,0,Grocery Store,Park,Food Truck,Bank,Liquor Store,Bus Stop,Hotel,Baseball Field,Shopping Mall,Discount Store


In [198]:
northyork_merged.loc[northyork_merged['Cluster_Labels'] == 1, northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,1,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Women's Store,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
4,Glencairn,1,Pizza Place,Sushi Restaurant,Bakery,Japanese Restaurant,Pub,Women's Store,Department Store,Chocolate Shop,Clothing Store,Coffee Shop
15,Humber Summit,1,Furniture / Home Store,Pizza Place,Intersection,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


In [199]:
northyork_merged.loc[northyork_merged['Cluster_Labels'] == 2, northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"Willowdale, Newtonbrook",2,Park,Dessert Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
22,York Mills West,2,Park,Convenience Store,Dessert Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop


In [200]:
northyork_merged.loc[northyork_merged['Cluster_Labels'] == 3, northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"Humberlea, Emery",3,Baseball Field,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [201]:
northyork_merged.loc[northyork_merged['Cluster_Labels'] == 4, northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,4,Park,Food & Drink Shop,Fast Food Restaurant,Department Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping


In [203]:
northyork_merged['1st Most Common Venue'].value_counts()

Grocery Store             4
Park                      4
Furniture / Home Store    2
Clothing Store            2
Gym                       2
Bank                      1
Fast Food Restaurant      1
Ramen Restaurant          1
Pharmacy                  1
Japanese Restaurant       1
Baseball Field            1
Italian Restaurant        1
Pizza Place               1
Coffee Shop               1
Name: 1st Most Common Venue, dtype: int64

Most common venues in North York are Parks and Grocery stores